You need to implement Logistic Regression from scratch in this question

1. You are provided with the dataset of sign language digits. Implement logistic regression from scratch to classify the images provided in the dataset. Load the dataset and perform splitting into training and test sets with 70:30 ratio randomly using test train split.
2. Plot a diagram for the sigmoid function. This is used for binary classi cation. How do you modify it for multilabel dataset classification problems? State and Explain the methods used.
3. Use both one vs all and one vs one method for the above problem statement purpose.
4. Also get results using Log Reg from scikit learn.
5. Report accuracy score, Confusion matrix and any other metrics you feel useful and Compare the results - from all the three.



[BONUS]
6. Display few pictures with their predicted and original labels 
7. Do the results differ? State the reasons why it is so.


dataset link : 
https://iiitaphyd-my.sharepoint.com/:f:/g/personal/apurva_jadhav_students_iiit_ac_in/Eictt5_qmoxNqezgQQiMWeIBph4sxlfA6jWAJNPnV2SF9Q?e=mQmYN0 

                

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib as plt

In [3]:
x_l = np.load("X.npy") # image
y_l = np.load("Y.npy") # label


In [4]:
x_l.shape

(2062, 64, 64)

In [5]:
y_l.shape

(2062, 10)

In [6]:
y_l[0]

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [14]:
sigmoid = lambda x : 1/(1 + np.exp(-x))

In [ ]:
# Linear Regression
class LinearRegression:
    def __init__(self, train_data, Y):
        self.data = train_data  # It is assumed that data is normalized and shuffled (rows, cols)
        self.Y = Y[:, np.newaxis]
        self.b = np.random.randn()
        print(self.data.shape)
        self.cols = self.data.shape[1]
        self.rows = self.data.shape[0]
        self.weights = np.random.randn(self.cols, 1)  # Initialising weights to 1, shape (cols, 1)
        self.num_iterations = 100
        self.learning_rate = 0.0001
        self.batch_size = 50
    
    @staticmethod
    def sigmoid(x):
        return 1/(1 + np.exp(-x))
        
    def calc_mini_batches(self):
        new_data = np.hstack((self.data, self.Y))
        np.random.shuffle(new_data)
    
        rem = self.rows % self.batch_size
        num = self.rows // self.batch_size
        till = self.batch_size * num
        if num > 0:
            dd = np.array(np.vsplit(new_data[ :till, :], num))
            X_batch = dd[:, :, :-1]
            Y_batch = dd[:, :, -1]
        
        if rem != 0:
            curr_batch = new_data[till: , :]
            diff = self.batch_size - rem
            temp = np.zeros((diff, self.cols+1)) # appending with zeros so that the sizes can be matched
            curr_batch = np.concatenate((curr_batch, temp))[np.newaxis, :, :]
            if num > 0:
                X_batch = np.concatenate((X_batch, curr_batch[:, :, :-1]))
                Y_batch = np.concatenate((Y_batch, curr_batch[:, :, -1]))
            else:
                X_batch = curr_batch[:, :, :-1]
                Y_batch = curr_batch[:, :, -1]
            
        return X_batch, Y_batch

    def update_weights(self, X, Y):
        Y_predicted = np.dot(X, self.weights) + self.b  # Remember that X has data stored along the row for one sample
        Y_predicted = self.sigmoid(Y_predicted) 
        gradient = np.dot(np.transpose(X), Y_predicted - Y)
        self.b = self.b - np.sum(Y_predicted - Y)
        self.weights = self.weights - (self.learning_rate * gradient) # vector subtraction
    
    def print_error(self):
        Y_Predicted = self.sigmoid(self.predict(self.data))
        mask = self.Y == 1
        mask 
        val = np.sum((Y_Predicted - self.Y) ** 2) / self.rows
        print(val/1000)
    
    def gradient_descent(self):
        for j in range(self.num_iterations):
            X, Y = self.calc_mini_batches()
            num_batches = X.shape[0]
            for i in range(num_batches):
                self.update_weights(X[i, :, :], Y[i, :][:, np.newaxis])  # update the weights
            if j%20 == 0:
                self.print_error()
    
    def predict(self, X):
        # X is 2 dimensional array, samples along the rows
        return np.dot(X, self.weights) + self.b